# 松材线虫病知识图谱查询工具

## 📋 简介

本 Notebook 提供了一套完整的知识图谱查询工具，用于探索和展示松材线虫病知识图谱的成果。

### 🎯 主要功能

1. 📊 **数据库统计信息** - 查看整体规模和结构
2. 📈 **节点度数分布** - 识别核心概念
3. 🔍 **节点查询** - 按类型或关键词搜索
4. 🔗 **关系探索** - 查看节点间的连接
5. 🛤️ **路径分析** - 查找最短路径
6. 🌐 **社区分析** - 识别知识聚类
7. 📋 **知识三元组** - 结构化知识展示

### 💡 使用说明

- 按顺序运行单元格，或根据需要运行特定查询
- 修改查询参数以探索不同的数据
- 使用 Markdown 单元格记录分析发现

## 🔧 1. 环境准备

### 安装依赖包

In [1]:
# 安装必要的依赖包
!pip install neo4j tabulate colorama pandas

  Using cached neo4j-6.0.3-py3-none-any.whl.metadata (5.2 kB)
Using cached neo4j-6.0.3-py3-none-any.whl (325 kB)


### 导入库

In [2]:
import os
import sys
from typing import List, Dict, Any, Optional
from neo4j import GraphDatabase
from tabulate import tabulate
from colorama import init, Fore, Style
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 初始化 colorama
init(autoreset=True)

print(f"{Fore.GREEN}✓ 所有依赖库导入成功！{Style.RESET_ALL}")

✓ 所有依赖库导入成功！


## 🗄️ 2. 数据库连接

配置并连接到 Neo4j 数据库

In [3]:
# 数据库连接配置
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "12345678"

# 创建数据库连接
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def execute_query(query: str, parameters: Optional[Dict] = None) -> List[Dict]:
    """执行 Cypher 查询"""
    with driver.session() as session:
        result = session.run(query, parameters or {})
        return [dict(record) for record in result]

# 测试连接
try:
    test_result = execute_query("RETURN 1 as test")
    print(f"{Fore.GREEN}✓ 成功连接到 Neo4j 数据库！{Style.RESET_ALL}")
    print(f"{Fore.BLUE}ℹ 连接地址: {NEO4J_URI}{Style.RESET_ALL}")
except Exception as e:
    print(f"{Fore.RED}✗ 连接失败: {e}{Style.RESET_ALL}")

✓ 成功连接到 Neo4j 数据库！
ℹ 连接地址: bolt://localhost:7687


## 📊 3. 数据库统计信息

查看知识图谱的整体规模和结构

In [4]:
# 获取节点总数
node_count_query = "MATCH (n) RETURN count(n) as total_nodes"
node_result = execute_query(node_count_query)
total_nodes = node_result[0]['total_nodes']

# 获取关系总数
rel_count_query = "MATCH ()-[r]->() RETURN count(r) as total_relationships"
rel_result = execute_query(rel_count_query)
total_relationships = rel_result[0]['total_relationships']

# 打印基本统计
print(f"{Fore.CYAN}{'='*80}")
print(f"{Fore.CYAN}{'数据库统计信息':^70}")
print(f"{Fore.CYAN}{'='*80}{Style.RESET_ALL}\n")

print(f"{Fore.GREEN}总节点数: {Fore.WHITE}{total_nodes:,}")
print(f"{Fore.GREEN}总关系数: {Fore.WHITE}{total_relationships:,}")
print(f"{Fore.GREEN}平均度数: {Fore.WHITE}{total_relationships*2/total_nodes:.2f}")

                               数据库统计信息                                

总节点数: 155
总关系数: 216
平均度数: 2.79


### 节点类型分布

In [5]:
# 节点标签统计
label_query = """
MATCH (n)
WITH labels(n)[0] as label, count(*) as count
RETURN label, count
ORDER BY count DESC
"""
label_stats = execute_query(label_query)

# 创建 DataFrame
df_labels = pd.DataFrame(label_stats)
df_labels['percentage'] = (df_labels['count'] / total_nodes * 100).round(2)
df_labels['percentage'] = df_labels['percentage'].astype(str) + '%'

print(f"\n{Fore.CYAN}节点类型分布:")
print(tabulate(df_labels, headers=['节点类型', '数量', '占比'], 
               tablefmt='grid', showindex=False))

# 显示 DataFrame（更适合 Jupyter）
display(df_labels)


节点类型分布:
+------------+--------+--------+
| 节点类型   |   数量 | 占比   |
+============+========+========+
| Other      |     83 | 53.55% |
+------------+--------+--------+
| Host       |     19 | 12.26% |
+------------+--------+--------+
| Location   |     18 | 11.61% |
+------------+--------+--------+
| Technology |     12 | 7.74%  |
+------------+--------+--------+
| Vector     |      8 | 5.16%  |
+------------+--------+--------+
| Disease    |      8 | 5.16%  |
+------------+--------+--------+
| Pathogen   |      4 | 2.58%  |
+------------+--------+--------+
| Control    |      3 | 1.94%  |
+------------+--------+--------+


,label,count,percentage
0,Other,83,53.55%
1,Host,19,12.26%
2,Location,18,11.61%
3,Technology,12,7.74%
4,Vector,8,5.16%
5,Disease,8,5.16%
6,Pathogen,4,2.58%
7,Control,3,1.94%


### 关系类型分布

In [6]:
# 关系类型统计
rel_type_query = """
MATCH ()-[r]->()
WITH type(r) as rel_type, count(*) as count
RETURN rel_type, count
ORDER BY count DESC
"""
rel_type_stats = execute_query(rel_type_query)

# 创建 DataFrame
df_rels = pd.DataFrame(rel_type_stats)
df_rels['percentage'] = (df_rels['count'] / total_relationships * 100).round(2)
df_rels['percentage'] = df_rels['percentage'].astype(str) + '%'

print(f"\n{Fore.CYAN}关系类型分布:")
print(tabulate(df_rels, headers=['关系类型', '数量', '占比'], 
               tablefmt='grid', showindex=False))

# 显示 DataFrame
display(df_rels)


关系类型分布:
+----------------+--------+--------+
| 关系类型       |   数量 | 占比   |
+================+========+========+
| 影响           |     39 | 18.06% |
+----------------+--------+--------+
| 传播           |     38 | 17.59% |
+----------------+--------+--------+
| 防治           |     23 | 10.65% |
+----------------+--------+--------+
| 分布于         |     19 | 8.8%   |
+----------------+--------+--------+
| 引起           |     13 | 6.02%  |
+----------------+--------+--------+
| 寄生           |     13 | 6.02%  |
+----------------+--------+--------+
| 感染           |      6 | 2.78%  |
+----------------+--------+--------+
| 发生于         |      5 | 2.31%  |
+----------------+--------+--------+
| 分布           |      4 | 1.85%  |
+----------------+--------+--------+
| 抑制           |      4 | 1.85%  |
+----------------+--------+--------+
| 相关           |      4 | 1.85%  |
+----------------+--------+--------+
| 包含           |      3 | 1.39%  |
+----------------+--------+--------+
| 基于           |      3 | 

,rel_type,count,percentage
0,影响,39,18.06%
1,传播,38,17.59%
2,防治,23,10.65%
3,分布于,19,8.8%
4,引起,13,6.02%
5,寄生,13,6.02%
6,感染,6,2.78%
7,发生于,5,2.31%
8,分布,4,1.85%
9,抑制,4,1.85%


## 📈 4. 节点度数分布

查看连接最多的核心节点（Top 20）

In [7]:
query = """
MATCH (n)
OPTIONAL MATCH (n)-[r]-()
WITH n, count(r) as degree
WHERE degree > 0
RETURN n.name as node_name, 
       labels(n)[0] as node_type,
       degree
ORDER BY degree DESC
LIMIT 20
"""
results = execute_query(query)

if results:
    df_degree = pd.DataFrame(results)
    df_degree.insert(0, '排名', range(1, len(df_degree) + 1))
    
    print(f"{Fore.CYAN}{'='*80}")
    print(f"{Fore.CYAN}{'节点度数分布 (Top 20)':^70}")
    print(f"{Fore.CYAN}{'='*80}{Style.RESET_ALL}\n")
    
    print(tabulate(df_degree, headers=['排名', '节点名称', '类型', '度数'], 
                   tablefmt='grid', showindex=False))
    
    display(df_degree)
else:
    print(f"{Fore.YELLOW}⚠ 未找到相关数据{Style.RESET_ALL}")

                           节点度数分布 (Top 20)                            

+--------+----------------------------+----------+--------+
|   排名 | 节点名称                   | 类型     |   度数 |
+========+============================+==========+========+
|      1 | 松材线虫病                 | Disease  |     49 |
+--------+----------------------------+----------+--------+
|      2 | 松材线虫                   | Pathogen |     47 |
+--------+----------------------------+----------+--------+
|      3 | 气候变化                   | Other    |     18 |
+--------+----------------------------+----------+--------+
|      4 | 松褐天牛                   | Vector   |     15 |
+--------+----------------------------+----------+--------+
|      5 | 马尾松                     | Host     |     13 |
+--------+----------------------------+----------+--------+
|      6 | 萎蔫                       | Disease  |     10 |
+--------+----------------------------+----------+--------+
|      7 | 噻唑膦                     | Other    |      9 |
+--

,排名,node_name,node_type,degree
0,1,松材线虫病,Disease,49
1,2,松材线虫,Pathogen,47
2,3,气候变化,Other,18
3,4,松褐天牛,Vector,15
4,5,马尾松,Host,13
5,6,萎蔫,Disease,10
6,7,噻唑膦,Other,9
7,8,monochamus alternatus,Vector,8
8,9,云杉花墨天牛,Vector,8
9,10,bursaphelenchus xylophilus,Pathogen,8


## 🔍 5. 按类型查看节点

查看特定类型的节点。可用类型：
- **Pathogen**: 病原体
- **Host**: 寄主植物
- **Vector**: 传播媒介
- **Disease**: 病害症状
- **Location**: 地理位置
- **Control**: 防控措施
- **Environment**: 环境因素
- **Technology**: 检测技术
- **Other**: 其他概念

In [8]:
# 修改这里的参数
node_type = "Pathogen"  # 修改为你想查询的类型
limit = 10              # 显示数量

query = f"""
MATCH (n:{node_type})
OPTIONAL MATCH (n)-[r]-()
WITH n, count(r) as degree
RETURN n.name as name, 
       n.type as type,
       degree
ORDER BY degree DESC
LIMIT {limit}
"""
results = execute_query(query)

if results:
    df = pd.DataFrame(results)
    df.insert(0, '序号', range(1, len(df) + 1))
    
    print(f"{Fore.CYAN}{node_type} 类型节点（前 {limit} 个）{Style.RESET_ALL}\n")
    print(tabulate(df, headers=['序号', '名称', '子类型', '度数'], 
                   tablefmt='grid', showindex=False))
    display(df)
else:
    print(f"{Fore.YELLOW}⚠ 未找到 {node_type} 类型节点{Style.RESET_ALL}")

Pathogen 类型节点（前 10 个）

+--------+------------------------------------+----------+--------+
|   序号 | 名称                               | 子类型   |   度数 |
+========+====================================+==========+========+
|      1 | 松材线虫                           | Pathogen |     47 |
+--------+------------------------------------+----------+--------+
|      2 | bursaphelenchus xylophilus         | Pathogen |      8 |
+--------+------------------------------------+----------+--------+
|      3 | 松材线虫感染                       | Pathogen |      4 |
+--------+------------------------------------+----------+--------+
|      4 | 松材线虫bursaphelenchus xylophilus | Pathogen |      2 |
+--------+------------------------------------+----------+--------+


,序号,name,type,degree
0,1,松材线虫,Pathogen,47
1,2,bursaphelenchus xylophilus,Pathogen,8
2,3,松材线虫感染,Pathogen,4
3,4,松材线虫bursaphelenchus xylophilus,Pathogen,2


## 🔎 6. 搜索节点

根据关键词模糊搜索节点

In [9]:
# 修改搜索关键词
keyword = "线虫"  # 修改为你想搜索的关键词

query = """
MATCH (n)
WHERE toLower(n.name) CONTAINS toLower($keyword)
OPTIONAL MATCH (n)-[r]-()
WITH n, count(r) as degree
RETURN n.name as name,
       labels(n)[0] as type,
       degree
ORDER BY degree DESC
LIMIT 20
"""
results = execute_query(query, {'keyword': keyword})

if results:
    df = pd.DataFrame(results)
    df.insert(0, '序号', range(1, len(df) + 1))
    
    print(f"{Fore.CYAN}搜索结果: '{keyword}'{Style.RESET_ALL}\n")
    print(tabulate(df, headers=['序号', '名称', '类型', '度数'], 
                   tablefmt='grid', showindex=False))
    print(f"\n{Fore.GREEN}✓ 找到 {len(results)} 个匹配节点{Style.RESET_ALL}")
    display(df)
else:
    print(f"{Fore.YELLOW}⚠ 未找到包含 '{keyword}' 的节点{Style.RESET_ALL}")

搜索结果: '线虫'

+--------+--------------------------------------------+----------+--------+
|   序号 | 名称                                       | 类型     |   度数 |
+========+============================================+==========+========+
|      1 | 松材线虫病                                 | Disease  |     49 |
+--------+--------------------------------------------+----------+--------+
|      2 | 松材线虫                                   | Pathogen |     47 |
+--------+--------------------------------------------+----------+--------+
|      3 | 松材线虫病疫情                             | Disease  |      7 |
+--------+--------------------------------------------+----------+--------+
|      4 | 松材线虫病疫情指数                         | Disease  |      5 |
+--------+--------------------------------------------+----------+--------+
|      5 | 松材线虫感染                               | Pathogen |      4 |
+--------+--------------------------------------------+----------+--------+
|      6 | 松材线虫bursaphelenchus xylophil

,序号,name,type,degree
0,1,松材线虫病,Disease,49
1,2,松材线虫,Pathogen,47
2,3,松材线虫病疫情,Disease,7
3,4,松材线虫病疫情指数,Disease,5
4,5,松材线虫感染,Pathogen,4
5,6,松材线虫bursaphelenchus xylophilus,Pathogen,2
6,7,松材线虫病外部症状表现与病害治疗效果的关系,Disease,2


## 🔗 7. 查看节点关系

查看指定节点的所有关系

In [10]:
# 修改节点名称
node_name = "松材线虫"  # 修改为你想查询的节点

query = """
MATCH (n {name: $node_name})-[r]-(m)
RETURN n.name as source,
       type(r) as relationship,
       m.name as target,
       labels(m)[0] as target_type
LIMIT 50
"""
results = execute_query(query, {'node_name': node_name})

if results:
    df = pd.DataFrame(results)
    df.insert(0, '序号', range(1, len(df) + 1))
    
    print(f"{Fore.CYAN}节点关系: '{node_name}'{Style.RESET_ALL}\n")
    print(tabulate(df, headers=['序号', '源节点', '关系类型', '目标节点', '目标类型'], 
                   tablefmt='grid', showindex=False))
    print(f"\n{Fore.GREEN}✓ 找到 {len(results)} 条关系{Style.RESET_ALL}")
    display(df)
else:
    print(f"{Fore.YELLOW}⚠ 未找到节点 '{node_name}' 或该节点没有关系{Style.RESET_ALL}")

节点关系: '松材线虫'

+--------+----------+------------+--------------------------------------------+------------+
|   序号 | 源节点   | 关系类型   | 目标节点                                   | 目标类型   |
+========+==========+============+============================================+============+
|      1 | 松材线虫 | 感染       | 疫木真菌群落                               | Other      |
+--------+----------+------------+--------------------------------------------+------------+
|      2 | 松材线虫 | 感染       | 20年生黑松                                 | Host       |
+--------+----------+------------+--------------------------------------------+------------+
|      3 | 松材线虫 | 感染       | 马尾松                                     | Host       |
+--------+----------+------------+--------------------------------------------+------------+
|      4 | 松材线虫 | 引起       | 统性导管栓塞                               | Other      |
+--------+----------+------------+--------------------------------------------+------------+
|      5 | 松材线虫 | 引起   

,序号,source,relationship,target,target_type
0,1,松材线虫,感染,疫木真菌群落,Other
1,2,松材线虫,感染,20年生黑松,Host
2,3,松材线虫,感染,马尾松,Host
3,4,松材线虫,引起,统性导管栓塞,Other
4,5,松材线虫,引起,典范对应分析,Other
5,6,松材线虫,引起,松材线虫病,Disease
6,7,松材线虫,分布于,分布区,Location
7,8,松材线虫,分布于,疫区,Location
8,9,松材线虫,寄生,真菌,Other
9,10,松材线虫,寄生,细胞色素c血红素裂合酶基因(hccs),Other


## 📋 8. 按类型查看关系

查看特定类型的关系。常见关系类型：
- **感染** - 病原体感染寄主
- **引起** - 导致病害症状
- **传播** - 媒介传播
- **防治** - 防控措施
- **分布于** - 地理分布
- **同义** - 同义词

In [11]:
# 修改关系类型
rel_type = "感染"  # 修改为你想查询的关系类型
limit = 20

query = f"""
MATCH (n)-[r:{rel_type}]->(m)
RETURN n.name as source,
       labels(n)[0] as source_type,
       type(r) as relationship,
       m.name as target,
       labels(m)[0] as target_type
LIMIT {limit}
"""
results = execute_query(query)

if results:
    df = pd.DataFrame(results)
    df.insert(0, '序号', range(1, len(df) + 1))
    
    print(f"{Fore.CYAN}'{rel_type}' 类型关系（前 {limit} 条）{Style.RESET_ALL}\n")
    print(tabulate(df, headers=['序号', '源节点', '源类型', '关系', '目标节点', '目标类型'], 
                   tablefmt='grid', showindex=False))
    display(df)
else:
    print(f"{Fore.YELLOW}⚠ 未找到 '{rel_type}' 类型关系{Style.RESET_ALL}")

'感染' 类型关系（前 20 条）

+--------+------------------------------------+----------+--------+------------------------+------------+
|   序号 | 源节点                             | 源类型   | 关系   | 目标节点               | 目标类型   |
+========+====================================+==========+========+========================+============+
|      1 | 马尾松                             | Host     | 感染   | 松材线虫               | Pathogen   |
+--------+------------------------------------+----------+--------+------------------------+------------+
|      2 | 松材线虫                           | Pathogen | 感染   | 疫木真菌群落           | Other      |
+--------+------------------------------------+----------+--------+------------------------+------------+
|      3 | 松材线虫                           | Pathogen | 感染   | 20年生黑松             | Host       |
+--------+------------------------------------+----------+--------+------------------------+------------+
|      4 | bursaphelenchus xylophilus         | Pathogen | 感染   | pinus thun

,序号,source,source_type,relationship,target,target_type
0,1,马尾松,Host,感染,松材线虫,Pathogen
1,2,松材线虫,Pathogen,感染,疫木真菌群落,Other
2,3,松材线虫,Pathogen,感染,20年生黑松,Host
3,4,bursaphelenchus xylophilus,Pathogen,感染,pinus thunbergii,Host
4,5,松材线虫bursaphelenchus xylophilus,Pathogen,感染,马尾松pinus massoniana,Host
5,6,松材线虫病,Disease,感染,樟子松,Host


## 🛤️ 9. 最短路径查询

查找两个节点之间的最短连接路径

In [12]:
# 修改源节点和目标节点
source = "松材线虫"  # 源节点
target = "马尾松"    # 目标节点

query = """
MATCH path = shortestPath(
    (source {name: $source})-[*..10]-(target {name: $target})
)
RETURN [node in nodes(path) | node.name] as path_nodes,
       [rel in relationships(path) | type(rel)] as path_rels,
       length(path) as path_length
"""
results = execute_query(query, {'source': source, 'target': target})

if results:
    for i, r in enumerate(results):
        print(f"\n{Fore.CYAN}最短路径: '{source}' → '{target}'{Style.RESET_ALL}\n")
        print(f"{Fore.GREEN}路径 {i+1}:")
        print(f"{Fore.CYAN}长度: {Fore.WHITE}{r['path_length']}\n")
        
        # 打印路径
        nodes = r['path_nodes']
        rels = r['path_rels']
        
        path_str = ""
        for j, node in enumerate(nodes):
            path_str += f"{Fore.YELLOW}{node}{Style.RESET_ALL}"
            if j < len(rels):
                path_str += f" {Fore.BLUE}--[{rels[j]}]-->{Style.RESET_ALL} "
        
        print(path_str)
        
        # 创建路径 DataFrame
        path_data = []
        for j in range(len(nodes)):
            if j < len(rels):
                path_data.append([j+1, nodes[j], rels[j], nodes[j+1]])
        
        if path_data:
            df_path = pd.DataFrame(path_data, columns=['步骤', '起点', '关系', '终点'])
            print("\n路径详情:")
            display(df_path)
else:
    print(f"{Fore.YELLOW}⚠ 未找到从 '{source}' 到 '{target}' 的路径{Style.RESET_ALL}")


最短路径: '松材线虫' → '马尾松'

路径 1:
长度: 1

松材线虫 --[感染]--> 马尾松

路径详情:


,步骤,起点,关系,终点
0,1,松材线虫,感染,马尾松


## 🌐 10. 核心节点群落分析

分析连接紧密的核心节点

In [13]:
query = """
MATCH (n)
OPTIONAL MATCH (n)-[r]-(m)
WITH n, count(DISTINCT m) as neighbors, count(r) as total_connections
WHERE neighbors >= 3
RETURN n.name as node_name,
       labels(n)[0] as node_type,
       neighbors,
       total_connections
ORDER BY total_connections DESC
LIMIT 20
"""
results = execute_query(query)

if results:
    df = pd.DataFrame(results)
    df.insert(0, '排名', range(1, len(df) + 1))
    
    print(f"{Fore.CYAN}{'='*80}")
    print(f"{Fore.CYAN}{'核心节点群落分析':^70}")
    print(f"{Fore.CYAN}{'='*80}{Style.RESET_ALL}\n")
    
    print(tabulate(df, headers=['排名', '节点名称', '类型', '邻居数', '总连接数'], 
                   tablefmt='grid', showindex=False))
    display(df)
else:
    print(f"{Fore.YELLOW}⚠ 未找到核心节点群落{Style.RESET_ALL}")

                               核心节点群落分析                               

+--------+----------------------------+----------+----------+------------+
|   排名 | 节点名称                   | 类型     |   邻居数 |   总连接数 |
+========+============================+==========+==========+============+
|      1 | 松材线虫病                 | Disease  |       46 |         49 |
+--------+----------------------------+----------+----------+------------+
|      2 | 松材线虫                   | Pathogen |       43 |         47 |
+--------+----------------------------+----------+----------+------------+
|      3 | 气候变化                   | Other    |       17 |         18 |
+--------+----------------------------+----------+----------+------------+
|      4 | 松褐天牛                   | Vector   |       10 |         15 |
+--------+----------------------------+----------+----------+------------+
|      5 | 马尾松                     | Host     |       12 |         13 |
+--------+----------------------------+----------+----------+--

,排名,node_name,node_type,neighbors,total_connections
0,1,松材线虫病,Disease,46,49
1,2,松材线虫,Pathogen,43,47
2,3,气候变化,Other,17,18
3,4,松褐天牛,Vector,10,15
4,5,马尾松,Host,12,13
5,6,萎蔫,Disease,10,10
6,7,噻唑膦,Other,8,9
7,8,monochamus alternatus,Vector,8,8
8,9,bursaphelenchus xylophilus,Pathogen,8,8
9,10,云南松,Host,7,8


## 📄 11. 知识三元组展示

查看知识图谱的三元组表示（主语-谓语-宾语）

In [14]:
# 修改显示数量
limit = 20

query = f"""
MATCH (n)-[r]->(m)
RETURN n.name as subject,
       labels(n)[0] as subject_type,
       type(r) as predicate,
       m.name as object,
       labels(m)[0] as object_type
LIMIT {limit}
"""
results = execute_query(query)

if results:
    df = pd.DataFrame(results)
    df.insert(0, '序号', range(1, len(df) + 1))
    
    print(f"{Fore.CYAN}知识三元组示例（前 {limit} 条）{Style.RESET_ALL}\n")
    print(tabulate(df, headers=['序号', '主语', '主语类型', '谓语', '宾语', '宾语类型'], 
                   tablefmt='grid', showindex=False))
    display(df)
else:
    print(f"{Fore.YELLOW}⚠ 未找到知识三元组{Style.RESET_ALL}")

知识三元组示例（前 20 条）

+--------+----------+------------+--------+---------------------------------+------------+
|   序号 | 主语     | 主语类型   | 谓语   | 宾语                            | 宾语类型   |
+========+==========+============+========+=================================+============+
|      1 | 马尾松   | Host       | 感染   | 松材线虫                        | Pathogen   |
+--------+----------+------------+--------+---------------------------------+------------+
|      2 | 马尾松   | Host       | 引起   | 萎蔫                            | Disease    |
+--------+----------+------------+--------+---------------------------------+------------+
|      3 | 马尾松   | Host       | 引起   | 典范对应分析                    | Other      |
+--------+----------+------------+--------+---------------------------------+------------+
|      4 | 马尾松   | Host       | 寄生   | 松褐天牛                        | Vector     |
+--------+----------+------------+--------+---------------------------------+------------+
|      5 | 马尾松   | Host       | 同义

,序号,subject,subject_type,predicate,object,object_type
0,1,马尾松,Host,感染,松材线虫,Pathogen
1,2,马尾松,Host,引起,萎蔫,Disease
2,3,马尾松,Host,引起,典范对应分析,Other
3,4,马尾松,Host,寄生,松褐天牛,Vector
4,5,马尾松,Host,同义,云南松,Host
5,6,马尾松,Host,同义,湿地松,Host
6,7,松材线虫,Pathogen,感染,疫木真菌群落,Other
7,8,松材线虫,Pathogen,感染,20年生黑松,Host
8,9,松材线虫,Pathogen,引起,统性导管栓塞,Other
9,10,松材线虫,Pathogen,引起,典范对应分析,Other


## 🎨 12. 自定义查询

在这里编写你自己的 Cypher 查询

In [15]:
# 自定义 Cypher 查询
custom_query = """
MATCH (n:Pathogen)-[r]->(m)
RETURN n.name as pathogen, type(r) as relation, m.name as target, labels(m)[0] as target_type
LIMIT 10
"""

results = execute_query(custom_query)

if results:
    df = pd.DataFrame(results)
    print(f"{Fore.CYAN}自定义查询结果:{Style.RESET_ALL}\n")
    display(df)
else:
    print(f"{Fore.YELLOW}⚠ 查询无结果{Style.RESET_ALL}")

自定义查询结果:



,pathogen,relation,target,target_type
0,松材线虫,感染,疫木真菌群落,Other
1,松材线虫,感染,20年生黑松,Host
2,松材线虫,引起,统性导管栓塞,Other
3,松材线虫,引起,典范对应分析,Other
4,松材线虫,分布于,分布区,Location
5,松材线虫,分布于,疫区,Location
6,松材线虫,寄生,真菌,Other
7,松材线虫,寄生,细胞色素c血红素裂合酶基因(hccs),Other
8,松材线虫,显示,光谱响应特征,Technology
9,松材线虫,使用,分类模型,Other


## 🔚 13. 关闭数据库连接

使用完毕后记得关闭连接

In [16]:
# 关闭数据库连接
driver.close()
print(f"{Fore.GREEN}✓ 数据库连接已关闭{Style.RESET_ALL}")
print(f"{Fore.CYAN}感谢使用松材线虫病知识图谱查询工具！{Style.RESET_ALL}")

✓ 数据库连接已关闭
感谢使用松材线虫病知识图谱查询工具！


---

## 📚 使用提示

### 🎯 快速开始
1. 按顺序运行前3个代码单元格（安装依赖、导入库、连接数据库）
2. 运行感兴趣的查询单元格
3. 修改查询参数探索不同的数据

### 💡 最佳实践
- 先运行统计信息了解整体情况
- 根据节点类型和关系类型进行针对性查询
- 使用搜索功能快速定位感兴趣的概念
- 通过路径查询探索概念之间的关联

### 🔧 自定义查询
- 使用第12节的自定义查询编写自己的 Cypher 语句
- 参考前面的查询示例学习 Cypher 语法
- 使用 `display(df)` 以表格形式展示结果

### 📊 数据导出
```python
# 将查询结果导出为 CSV
df.to_csv('query_results.csv', index=False, encoding='utf-8-sig')

# 导出为 Excel
df.to_excel('query_results.xlsx', index=False)
```

### 🎨 可视化建议
可以结合以下库进行可视化：
- **matplotlib/seaborn**: 统计图表
- **networkx**: 网络图
- **plotly**: 交互式图表

---

**💡 提示**: 修改每个查询单元格中的参数（如 `node_type`、`keyword`、`limit` 等）来探索不同的数据！